# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [552]:
DATA_FOLDER = 'Data' # Use the data folder provided in Tutorial 02 - Intro to Pandas.

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

In [553]:
import pandas as pd
import numpy as np
import glob
from datetime import date,time,datetime
import time
import xlrd
# Write your answer here

In [554]:
#input the data of guinea
path = r'Data\ebola\guinea_data'
allFiles = glob.glob(path + "/*.csv")
frame1 = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None,header=0)
    list_.append(df)
frame1 = pd.concat(list_)

path = r'Data\ebola\liberia_data'
allFiles = glob.glob(path + "/*.csv")
frame2 = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None,header=0)
    list_.append(df)
frame2 = pd.concat(list_)

path = r'Data\ebola\sl_data'
allFiles = glob.glob(path + "/*.csv")
frame3 = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None,header=0)
    list_.append(df)
frame3 = pd.concat(list_)

In [555]:
#rename the "Description" colunmn in order to make the same columns for the three countries
frame1 = frame1.rename(index=str, columns={"Description": "Variable"})
frame3 = frame3.rename(index=str, columns={"variable": "Variable","date": "Date"})

In [556]:
#add the tap of country
frame1['Country'] = 'guinea'
frame2['Country'] = 'liberia'
frame3['Country'] = 'sl'

In [557]:
#in order to combine the datas of three countries, we make them the same 'Variable'
frame1.rename(index=str, columns={"Totals": "National"},inplace=True)
frame2.replace('Total confirmed cases','Total cases of confirmed',inplace=True)
frame2.replace('Total probable cases','Total cases of probables',inplace=True)
frame2.replace('Total suspected cases','Total cases of suspects',inplace=True)
frame2.replace('Total death/s in confirmed cases','Total deaths of confirmed',inplace=True)
frame2.replace('Total death/s in probable cases','Total deaths of probables',inplace=True)
frame2.replace('Total death/s in suspected cases','Total deaths of suspects',inplace=True)

frame3.replace('cum_confirmed','Total cases of confirmed',inplace=True)
frame3.replace('cum_probable','Total cases of probables',inplace=True)
frame3.replace('cum_suspected','Total cases of suspects',inplace=True)
frame3.replace('death_confirmed','Total deaths of confirmed',inplace=True)
frame3.replace('death_probable','Total deaths of probables',inplace=True)
frame3.replace('death_suspected','Total deaths of suspects',inplace=True)

In [558]:
#since we only need to calculate data for each countriy, so we chose the data we want.
frame1 = frame1[['Date','Country','Variable','National']]
frame2 = frame2[['Date','Country','Variable','National']]
frame3 = frame3[['Date','Country','Variable','National']]

In [559]:
#merge data of three countries
DF = pd.merge(frame1,frame2,how='outer')
df = pd.merge(DF,frame3,how='outer')

In [560]:
#transfer Date data from str to datetime 
df['Date'] = pd.to_datetime(df.Date)

In [561]:
#get the data about 'Total cases' and 'Total deaths'. In my opinion, the other data such as 'New cases of suspects' are not necessary data for 
#us to calculate daily average per month of new cases and deaths, since data with 'Total' can truely reflect the situation while the 'New' can
#only reflect specific days' situation rather than per month.
sub1 = df[[name.startswith('Total cases of') for name in df.Variable]]
sub2 = df[[name.startswith('Total deaths of') for name in df.Variable]]

In [562]:
# merge the dataframe into a single dataframe
Data1 = pd.merge(sub1,sub2,how='outer')

In [563]:
Data= Data1.set_index(['Country','Variable','Date']).sort_index()

In [564]:
#delete the NAN 
Data = Data.dropna()

In [565]:
#some value of 'National' are strings like 1,276, we should transfer them to 1276
Data.loc['sl','Total cases of confirmed','2014--09-06']=1276
Data.loc['sl','Total cases of confirmed','2014--09-17']=1585
Data.loc['sl','Total cases of confirmed','2014--09-18']=1618
Data.loc['sl','Total cases of confirmed','2014--09-20']=1696
Data.loc['sl','Total cases of confirmed','2014--09-027']=2000

In [567]:
Str = ['guinea','liberia','sl']
String = ['Total cases of confirmed','Total cases of probables','Total cases of suspects','Total deaths of confirmed','Total deaths of probables','Total deaths of suspects']

In [591]:
Ave = np.zeros([3,6])
for i in range(0,3):
    for j in range(0,6):
        Lenth = Data.loc[Str[i],String[j]].shape[0]
        harf_lenth = int(Lenth/2)
        sub = Data.loc[Str[i],String[j]] #choose the data we wanted
        sub['National'] = pd.to_numeric(sub.National)#since values of 'National' are strings, we should transfer them into int or float
        sub1= sub[:harf_lenth]#we divide the data into two groups so that we can make full use of each data
        sub2 =sub[harf_lenth:2*harf_lenth]
        sub1= sub1.reset_index()
        sub2= sub2.reset_index()
        diff = sub2 - sub1 #For example we have 48 data for National adn we use 25th data to subtranct 1st data and then Divided by the time difference
        ave = np.zeros([harf_lenth,1])
        for t in range (0,harf_lenth):
            ave[t] = diff.National[t]/diff.Date[t].days
        Ave[i][j] = sum(ave)/(harf_lenth)        

In [592]:
AVE = pd.DataFrame(Ave)

In [600]:
AVE.columns = String
AVE.index=[Str]
AVE

,Total cases of confirmed,Total cases of probables,Total cases of suspects,Total deaths of confirmed,Total deaths of probables,Total deaths of suspects
guinea,12.316357,0.854977,-0.080584,6.980914,2.496392,1.073234
liberia,19.585189,15.020251,22.360379,10.798210,7.976506,5.558687
sl,49.146334,0.451809,10.297422,12.371717,2.711003,2.176776


## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

In [609]:
#input data and set the colunms
path = r'Data\microbiome'
allFiles = glob.glob(path + "/*.xls")
dfs = pd.DataFrame()
list_= []
i = 1
for file_ in allFiles[1:10]:
    df= pd.read_excel(file_, 'Sheet 1', index_col=None, header=None)
    String = 'MID'+str(i)
    df['BARCODE'] =String
    i = i + 1
    dfs = pd.concat([dfs,df],axis=0)
dfs.columns = ['Taxon','Count','BARCODE']

In [610]:
X_file = pd.ExcelFile(allFiles[0])
Df=X_file.parse('Sheet1')

In [605]:
#add the metadata information from the 10th spreadsheet as columns in the combined DataFrame
Data = pd.merge(dfs,Df,on='BARCODE',right_index = True)

In [611]:
#all the NaN values have been replaced by the tag unknown
Data.fillna('unknown')

,Taxon,Count,BARCODE,GROUP,SAMPLE
0,"Archaea ""Crenarchaeota"" Thermoprotei Desulfuro...",7,MID1,EXTRACTION CONTROL,unknown
1,"Archaea ""Crenarchaeota"" Thermoprotei Desulfuro...",2,MID1,EXTRACTION CONTROL,unknown
2,"Archaea ""Crenarchaeota"" Thermoprotei Sulfoloba...",3,MID1,EXTRACTION CONTROL,unknown
3,"Archaea ""Crenarchaeota"" Thermoprotei Thermopro...",3,MID1,EXTRACTION CONTROL,unknown
4,"Archaea ""Euryarchaeota"" ""Methanomicrobia"" Meth...",7,MID1,EXTRACTION CONTROL,unknown


In [608]:
Data.set_index(['Taxon','GROUP','SAMPLE']).sort_index()

Count  \
Taxon                                              GROUP              SAMPLE          
Archaea "Crenarchaeota" Thermoprotei Acidilobal... Control 1          tissue      1   
                                                   Control 2          tissue      5   
                                                   NEC 1              tissue      2   
Archaea "Crenarchaeota" Thermoprotei Acidilobal... Control 1          stool       1   
                                                                      tissue     15   
                                                   Control 2          stool       1   
                                                                      tissue     26   
                                                   NEC 1              tissue     14   
Archaea "Crenarchaeota" Thermoprotei Desulfuroc... Control 1          stool       8   
                                                                      tissue     14   
                                                   Control 2          stool      16   
                                                                      tissue     28   
                                                   EXTRACTION CONTROL NaN         7   
                                                   NEC 1              stool       7   
                                                                      tissue     23   
                                                   NEC 2              tissue      2   
Archaea "Crenarchaeota" Thermoprotei Desulfuroc... Control 2          tissue      1   
Archaea "Crenarchaeota" Thermoprotei Desulfuroc... Control 1          stool       2   
                                                                      tissue      4   
                                                   Control 2          stool       2   
                                                                      tissue      5   
                                                   NEC 1              stool       1   
                                                                      tissue      1   
Archaea "Crenarchaeota" Thermoprotei Desulfuroc... Control 1          tissue      1   
Archaea "Crenarchaeota" Thermoprotei Desulfuroc... Control 1          stool       1   
                                                                      tissue      1   
                                                   Control 2          tissue      2   
                                                   NEC 1              tissue      2   
Archaea "Crenarchaeota" Thermoprotei Desulfuroc... NEC 1              tissue      1   
Archaea "Crenarchaeota" Thermoprotei Desulfuroc... Control 1          stool       1   
...                                                                             ...   
Bacteria "Verrucomicrobia" Subdivision3   Subdi... Control 1          stool       4   
Bacteria "Verrucomicrobia" Verrucomicrobiae Ver... Control 1          stool       9   
                                                                      tissue      2   
                                                   NEC 1              stool       1   
Bacteria "Verrucomicrobia" Verrucomicrobiae Ver... Control 1          stool      39   
                                                                      tissue      1   
                                                   NEC 1              stool       1   
Bacteria Cyanobacteria Cyanobacteria  Chloropla... Control 1          stool       1   
                                                                      tissue      1   
                                                   Control 2          tissue      2   
                                                   EXTRACTION CONTROL NaN         2   
                                                   NEC 2              stool       1   
Bacteria Cyanobacteria Cyanobacteria  Chloropla... Control 1          stool       2   
                                                                      tissue

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.